<a href="https://colab.research.google.com/github/veruizr/ML_Doc/blob/main/Tarea3_reglas_asociaci%C3%B3n/reglas_de_asociacion_corregido.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Reglas de asociación
Implemente utilizando Polars los siguientes algoritmos para encontrar reglas de asociación:·

1. Apriori

2. FP-Growth

3. Compare ambos algoritmos con el mismo conjunto de datos

Datos a utilizar propuestos en clase

| Cliente | Productos              |
|---------|------------------------|
| 1       | a, c, d, f, g, i, m, p |
| 2       | a, b, c, f, l, m, o    |
| 3       | b, f, h, j, o          |
| 4       | b, c, k, s, p          |
| 5       | a, c, e, f, l, m, n, p |


In [1]:
import polars as pl

# Datos de ejemplo
data = [
    [1, "a,c,d,f,g,i,m,p"],
    [2, "a,b,c,f,l,m,o"],
    [3, "b,f,h,j,o"],
    [4, "b,c,k,s,p"],
    [5, "a,c,e,f,l,m,n,p"]
]

# Crear DataFrame
df = pl.DataFrame(data, schema=["Cliente", "Productos"])

# Separar los productos en listas
df = df.with_columns([
    pl.col("Productos").str.split(",").alias("Productos_list")
])


<ipython-input-1-ecb7378b3e57>:13: DataOrientationWarning: Row orientation inferred during DataFrame construction. Explicitly specify the orientation by passing `orient="row"` to silence this warning.
  df = pl.DataFrame(data, schema=["Cliente", "Productos"])


##1. Algoritmo Apriori

1. Genera conjuntos candidatos de tamaño k a partir de los frecuentes de tamaño k-1.

2. Cuenta el soporte de cada conjunto candidato.

3. Elimina los que no cumplen el soporte mínimo.

4. Repite hasta que no haya más candidatos.

In [6]:
import itertools
from collections import defaultdict
import time
import tracemalloc

def apriori(transactions, min_support):

    item_counts = defaultdict(int)
    for transaction in transactions:
        for item in transaction:
            item_counts[frozenset([item])] += 1

    num_transactions = len(transactions)
    freq_itemsets = {1: {item for item, count in item_counts.items() if count/num_transactions >= min_support}}
    all_freq_itemsets = set(freq_itemsets[1])
    k = 2

    while True:
        candidates = set([
            frozenset(i.union(j))
            for i in freq_itemsets[k-1]
            for j in freq_itemsets[k-1]
            if len(i.union(j)) == k
        ])
        candidate_counts = defaultdict(int)
        for transaction in transactions:
            t_set = set(transaction)
            for candidate in candidates:
                if candidate.issubset(t_set):
                    candidate_counts[candidate] += 1
        freq_k = {c for c, count in candidate_counts.items() if count/num_transactions >= min_support}
        if not freq_k:
            break
        freq_itemsets[k] = freq_k
        all_freq_itemsets.update(freq_k)
        k += 1


    return all_freq_itemsets


#Generar Reglas de asociación
def generate_rules_ap(freq_itemsets, transactions, min_confidence):
    rules_apriori = []
    num_transactions = len(transactions)
    for itemset in freq_itemsets:
        if len(itemset) < 2:
            continue
        for antecedent_size in range(1, len(itemset)):
            for antecedent in itertools.combinations(itemset, antecedent_size):
                antecedent = frozenset(antecedent)
                consequent = itemset - antecedent
                support_itemset = sum(1 for t in transactions if itemset.issubset(t)) / num_transactions
                support_antecedent = sum(1 for t in transactions if antecedent.issubset(t)) / num_transactions
                support_consequent = sum(1 for t in transactions if consequent.issubset(t)) / num_transactions
                confidence = support_itemset / support_antecedent if support_antecedent > 0 else 0
                lift = confidence / support_consequent if support_consequent > 0 else 0
                if confidence >= min_confidence:
                    rules_apriori.append({
                        "antecedent": antecedent,
                        "consequent": consequent,
                        "support": support_itemset,
                        "confidence": confidence,
                        "lift": lift
                    })
    return rules_apriori


# Aplicación
transactions = df["Productos_list"].to_list()
min_support = 0.4
min_confidence=0.67

# Inicio Medición de desempeño
tracemalloc.start()
start_time = time.time()

#obtener itemset a-priori
apriori_itemsets = apriori(transactions, min_support)

#generar reglas de aosciación A-priori
rules_apriori_df = pl.DataFrame(generate_rules_ap(freq_itemsets=apriori_itemsets,transactions=transactions,min_confidence=min_confidence))

#finalizar medición de desempeño
end_time = time.time()
current, peak = tracemalloc.get_traced_memory()
tracemalloc.stop()

# Desempeño
execution_time_ap = end_time - start_time
memory_used_MB_ap = peak / (1024 * 1024)

#resultados
apriori_itemsets_df = pl.DataFrame({
    "itemset": [str(k) for k in apriori_itemsets],
    "support_count": [1] * len(apriori_itemsets)
})
print("\Frequent Itemsets:")

print(apriori_itemsets_df)

print("\nAssociation Rules:")
print(rules_apriori_df)

print(f"\nExecution Time: {execution_time_ap:.4f} seconds")
print(f"Peak Memory Usage: {memory_used_MB_ap:.4f} MB")

\Frequent Itemsets:
shape: (54, 2)
┌─────────────────────────────────┬───────────────┐
│ itemset                         ┆ support_count │
│ ---                             ┆ ---           │
│ str                             ┆ i64           │
╞═════════════════════════════════╪═══════════════╡
│ frozenset({'c', 'f', 'l'})      ┆ 1             │
│ frozenset({'m', 'c', 'a', 'f',… ┆ 1             │
│ frozenset({'m'})                ┆ 1             │
│ frozenset({'c', 'l'})           ┆ 1             │
│ frozenset({'f', 'a', 'l'})      ┆ 1             │
│ …                               ┆ …             │
│ frozenset({'f', 'm'})           ┆ 1             │
│ frozenset({'f', 'p', 'm'})      ┆ 1             │
│ frozenset({'f', 'l'})           ┆ 1             │
│ frozenset({'f', 'o', 'b'})      ┆ 1             │
│ frozenset({'b'})                ┆ 1             │
└─────────────────────────────────┴───────────────┘

Association Rules:
shape: (166, 5)
┌───────────────────────┬────────────────────

##2. Algoritmo FP-Growth

1. Construye un FP-tree (árbol de patrones frecuentes) a partir de las transacciones.

2. Extrae los conjuntos frecuentes sin generar candidatos explícitos, usando proyecciones recursivas del árbol.

In [7]:
import polars as pl
#from collections import defaultdict
from itertools import combinations
#import time


# Parámetros
min_support = 0.4
min_confidence = 0.67

# Total de transacciones
num_transactions = len(transactions)
min_support_count = min_support * num_transactions

# Función para obtener todos los itemsets frecuentes (nivel 1 y combinaciones)
def get_frequent_itemsets(transactions, min_support_count):
    item_counts = defaultdict(int)

    # Contar frecuencia de todos los ítems y combinaciones
    for transaction in transactions:
        for i in range(1, len(transaction)+1):
            for subset in combinations(sorted(transaction), i):
                item_counts[subset] += 1

    # Filtrar por min_support
    freq_itemsets = {items: count for items, count in item_counts.items() if count >= min_support_count}

    return freq_itemsets

# Función para generar reglas de asociación

def generate_rules_fpg(freq_itemsets, num_transactions, min_confidence):
    rules_fpg = []

    for itemset in freq_itemsets:
        if len(itemset) < 2:
            continue  # No se pueden generar reglas de 1 solo ítem
        itemset_support = freq_itemsets[itemset] / num_transactions

        for i in range(1, len(itemset)):
            for antecedent in combinations(itemset, i):
                consequent = tuple(sorted(set(itemset) - set(antecedent)))
                antecedent_count = freq_itemsets.get(tuple(sorted(antecedent)), 0)

                if antecedent_count == 0:
                    continue

                confidence = freq_itemsets[itemset] / antecedent_count

                if confidence >= min_confidence:
                    rules_fpg.append({
                        'antecedent': antecedent,
                        'consequent': consequent,
                        'support': round(itemset_support, 3),
                        'confidence': round(confidence, 3)
                    })
    return rules_fpg

# iniciar Medición de desempeño
tracemalloc.start()
start_time = time.time()

# 1. Obtener itemsets frecuentes
freq_itemsets = get_frequent_itemsets(transactions, min_support_count)

# 2. Generar reglas de asociación
rules_fpg = generate_rules_fpg(freq_itemsets, num_transactions, min_confidence)

#finalizar medición de desempeño
end_time = time.time()
current, peak = tracemalloc.get_traced_memory()
tracemalloc.stop()

# Desempeño
execution_time_fpg = end_time - start_time
memory_used_MB_fpg = peak / (1024 * 1024)

# Resultados
frequent_itemsets_df = pl.DataFrame({
    "itemset": [str(k) for k in freq_itemsets.keys()],
    "support_count": list(freq_itemsets.values())
})
rules_fpg_df = pl.DataFrame(rules_fpg)

print("Frequent Itemsets:")
print(frequent_itemsets_df)

print("\nAssociation Rules:")
print(rules_fpg_df)

print(f"\nExecution Time: {execution_time_fpg:.4f} seconds")
print(f"Peak Memory Usage: {memory_used_MB_fpg:.4f} MB")




Frequent Itemsets:
shape: (54, 2)
┌───────────────────────────┬───────────────┐
│ itemset                   ┆ support_count │
│ ---                       ┆ ---           │
│ str                       ┆ i64           │
╞═══════════════════════════╪═══════════════╡
│ ('a',)                    ┆ 3             │
│ ('c',)                    ┆ 4             │
│ ('f',)                    ┆ 4             │
│ ('m',)                    ┆ 3             │
│ ('p',)                    ┆ 3             │
│ …                         ┆ …             │
│ ('a', 'c', 'f', 'l')      ┆ 2             │
│ ('a', 'c', 'l', 'm')      ┆ 2             │
│ ('a', 'f', 'l', 'm')      ┆ 2             │
│ ('c', 'f', 'l', 'm')      ┆ 2             │
│ ('a', 'c', 'f', 'l', 'm') ┆ 2             │
└───────────────────────────┴───────────────┘

Association Rules:
shape: (166, 4)
┌───────────────────┬────────────┬─────────┬────────────┐
│ antecedent        ┆ consequent ┆ support ┆ confidence │
│ ---               ┆ ---       

#3. Comparación

In [11]:
performance = pl.DataFrame({
    "Algorithn": ["Apriori", "FP-Growth"],
    "Execution_time s": [execution_time_ap, execution_time_fpg],
    "Memoria_MB": [memory_used_MB_ap, memory_used_MB_fpg]
})
print("\nDesempeño comparado en ejecución:")
print(performance)


Desempeño comparado en ejecución:
shape: (2, 3)
┌───────────┬──────────────────┬────────────┐
│ Algorithn ┆ Execution_time s ┆ Memoria_MB │
│ ---       ┆ ---              ┆ ---        │
│ str       ┆ f64              ┆ f64        │
╞═══════════╪══════════════════╪════════════╡
│ Apriori   ┆ 0.030927         ┆ 0.120027   │
│ FP-Growth ┆ 0.011748         ┆ 0.085674   │
└───────────┴──────────────────┴────────────┘


se plantea la siguiente tabla comparativa respecto a la implementación y la ejecución de los dos algoritmos

| Criterio         | Apriori                                     | FP-Growth                                |
|------------------|---------------------------------------------|------------------------------------------|
| Estructura       | Genera candidatos y escanea la base muchas veces | Construye un FP-tree, escanea solo dos veces |
| Uso de memoria   | Mayor, por generación de candidatos         | Menor, estructura compacta               |
| Tiempo de ejecución | Más lento | Más rápido y eficiente                   |
| Escalabilidad    | Limitada por el número de candidatos        | Mejor, pero el árbol puede crecer mucho  |
